# OsmGt example

## Imports and prepare input parameters

In [1]:
from IPython.display import display

from bokeh.plotting import output_notebook
from bokeh.plotting import show

import geopandas as gpd

from shapely.geometry import Point
from shapely.ops import linemerge

from gdf2bokeh import Gdf2Bokeh

from osmgt import OsmGt

from graph_tool.topology import shortest_path


output_notebook()


location = "Roanne"


Loading BokehJS ...

## Get POIs

In [2]:
%%time

pois_gdf = OsmGt.pois_from_location(location).get_gdf()

display(pois_gdf.head(2))

2021-04-21 17:30:17 - OsmGtPoi        - INFO     : From location: Roanne
2021-04-21 17:30:17 - OsmGtPoi        - INFO     : Loading data...
2021-04-21 17:30:17 - OsmGtPoi        - INFO     : NominatimApi: Query 200:OK in 0.24 sec.
2021-04-21 17:30:21 - OsmGtPoi        - INFO     : OverpassApi: Query 200:OK in 4.37 sec.
2021-04-21 17:30:21 - OsmGtPoi        - INFO     : Formating data
2021-04-21 17:30:21 - OsmGtPoi        - INFO     : Prepare GeoDataframe
a
b
c
d
2021-04-21 17:30:21 - OsmGtPoi        - INFO     : GeoDataframe Ready


,addr:postcode,amenity,atm,change_machine,name,opening_hours,operator,phone,ref:FR:LaPoste,source,...,service:bicycle:repair,check_date:opening_hours,contact:email,contact:facebook,source:opening_hours,clothes,waste,office,second_hand,geometry
0,42300,post_office,yes,yes,Roanne Principal,"Mo,We-Fr 08:30-18:00; Tu 08:30-12:00,13:45-18:...",La Poste,3631,07916A,data.gouv.fr:LaPoste - 10/2012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.07225 46.04071)
1,NaN,place_of_worship,NaN,NaN,Chapelle Jean Puy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (4.07073 46.03766)


CPU times: user 319 ms, sys: 7.47 ms, total: 326 ms
Wall time: 4.93 s


## Get Roads

In [3]:
%%time
roads_initialized = OsmGt.roads_from_location(
    location,
    mode="vehicle",
    additional_nodes=pois_gdf
)
roads_gdf = roads_initialized.get_gdf()

display(roads_gdf.head(2))

2021-04-21 16:44:40 - OsmGtRoads      - INFO     : From location: Roanne
2021-04-21 16:44:40 - OsmGtRoads      - INFO     : Loading data...
2021-04-21 16:44:40 - OsmGtRoads      - INFO     : NominatimApi: Query 200:OK in 0.19 sec.
2021-04-21 16:44:44 - OsmGtRoads      - INFO     : OverpassApi: Query 200:OK in 4.15 sec.
2021-04-21 16:44:45 - OsmGtRoads      - INFO     : Rebuild network data
2021-04-21 16:44:45 - OsmGtRoads      - INFO     : Network cleaning...
2021-04-21 16:44:45 - OsmGtRoads      - INFO     : Starting: Adding new nodes on the network
2021-04-21 16:44:45 - OsmGtRoads      - INFO     : Find nearest line for each node
2021-04-21 16:44:46 - OsmGtRoads      - INFO     : Split line
2021-04-21 16:44:46 - OsmGtRoads      - INFO     : Topology lines checker: to add: 267, to split: 264
2021-04-21 16:44:46 - OsmGtRoads      - INFO     : Starting: Find intersections
2021-04-21 16:44:46 - OsmGtRoads      - INFO     : Done: Find intersections
2021-04-21 16:44:46 - OsmGtRoads      - 

,highway,lanes:forward,maxspeed,name,oneway,ref,id,osm_url,topo_uuid,topology,...,sidewalk:bicycle,name:etymology:wikidata,EntityHand,SubClasses,alt_name,turn:lanes,source:name,maxspeed:backward,maxspeed:forward,geometry
0,primary,2,50,Rue de Charlieu,yes,D 482,24035569,https://www.openstreetmap.org/way/24035569,1_forward,unchanged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (4.08544 46.05234, 4.08546 46.05248..."
1,primary,NaN,50,Rue de Charlieu,yes,D 482,24035570,https://www.openstreetmap.org/way/24035570,2_forward,unchanged,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (4.09087 46.06639, 4.09084 46.06622..."


CPU times: user 2.18 s, sys: 243 ms, total: 2.42 s
Wall time: 7.15 s


## Display roads and nodes

In [4]:
%%time
layers_to_add = [
#     {
#         "input_gdf": roads_gdf,
#         "legend": "roads",
#         "color": "black",
#     },
    {
        "input_gdf": pois_gdf,
        "legend": "POIs",
        "color": "blue",
        "size": 9
    },
]



my_map = Gdf2Bokeh(
    "My roads and POIs - from OsmGT (https://github.com/amauryval)",
    layers=layers_to_add
)
# print(dir(my_map))
# print(my_map._Gdf2Bokeh__BOKEH_LAYER_CONTAINERS)



show(my_map.figure)

GlyphRenderer(id=1147, glyph=Circle(id='1145', ...), ...)
{'color': 'blue', 'size': 9, 'legend_label': 'POIs'}
ColumnDataSource(id='1143', ...)
<bound method Figure.circle of Figure(id='1110', ...)>


CPU times: user 361 ms, sys: 7.3 ms, total: 368 ms
Wall time: 401 ms


## Check topology details

In [ ]:
%%time

roads_topology_gdfs = roads_initialized.topology_checker()

lines_unchanged = roads_topology_gdfs["lines_unchanged"]
lines_added = roads_topology_gdfs["lines_added"]
lines_split = roads_topology_gdfs["lines_split"]
nodes_added = roads_topology_gdfs["nodes_added"]
intersections_added = roads_topology_gdfs["intersections_added"]

layers_to_add = [
    {
        "input_gdf": lines_unchanged,
        "legend": "roads unchanged",
        "color": "black",
    },
    {
        "input_gdf": lines_added,
        "legend": "roads added",
        "color": "green",
    },
    {
        "input_gdf": lines_split,
        "legend": "roads split",
        "color": "orange",
    },
    {
        "input_gdf": intersections_added,
        "legend": "Intersections added",
        "color": "brown",
    },
    {
        "input_gdf": nodes_added,
        "legend": "Nodes added",  # POIs here
        "style": "square",
        "color": "blue",
        "size": 9
    },
]

my_map = Gdf2Bokeh(
    "Topology about my roads and POIs - from OsmGT (https://github.com/amauryval)",
    layers=layers_to_add
)
show(my_map.figure)

## Get the graph-tool network and display it

In [ ]:
%%time
graph = roads_initialized.get_graph()

# a plot method has been added on OsmGT.
graph.plot()

## Compute a shortest path 

### With graph-tools function

In [ ]:
%%time
# now, we have to define a start point and a end point and get their wkt
start_node_topo_uuid = 47
end_node_topo_uuid = 63

# 'topo_uuid' is generated by osmgt (during the topology processing).
# Some roads has been split that's whyso this id has been created.
start_node_gdf = pois_gdf[pois_gdf['topo_uuid'] == start_node_topo_uuid]
end_node_gdf = pois_gdf[pois_gdf['topo_uuid'] == end_node_topo_uuid]

start_node_wkt = start_node_gdf.iloc[0]["geometry"].wkt
end_node_wkt = end_node_gdf.iloc[0]["geometry"].wkt

# the graph have some methods (graph-tools method always exists!) to find egdes, vertices... Let's use the .find_vertex_from_name(). the wkt is the vertex name...
source_vertex = graph.find_vertex_from_name(start_node_wkt)
target_vertex = graph.find_vertex_from_name(end_node_wkt)

# shortest path computing...
path_vertices, path_edges = shortest_path(
    graph,
    source=source_vertex,
    target=target_vertex,
    weights=graph.edge_weights  # weigth is based on line length
)

# get path by using edge names
roads_ids = [
    graph.edge_names[edge]
    for edge in path_edges
]

roads_gdf_copy = roads_gdf.copy(deep=True)
shortest_path_found = roads_gdf_copy[roads_gdf['topo_uuid'].isin(roads_ids)].to_crs(3857)['geometry'].to_list()
shortest_path_found_gdf = gpd.GeoDataFrame(index=[0], crs="EPSG:3857", geometry=[linemerge(shortest_path_found)])

layers_to_add = [
    {
        "input_gdf": shortest_path_found_gdf,
        "legend": "shortest_path",
        "color": "red",
        "line_width": 5
    },
    {
        "input_gdf": start_node_gdf,
        "legend": "source node",
        "color": "blue",
        "style": "circle",
        "size": 9
    },
    {
        "input_gdf": end_node_gdf,
        "legend": "target node",
        "color": "green",
        "style": "circle",
        "size": 9
    },
]

my_shortest_path_map = Gdf2Bokeh(
    "My shortest path - from OsmGT (https://github.com/amauryval)",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)

### With OsmGt function

In [ ]:
%%time

start_node_topo_uuid = 47
end_node_topo_uuid = 63

start_node_gdf = pois_gdf[pois_gdf['topo_uuid'] == start_node_topo_uuid]
end_node_gdf = pois_gdf[pois_gdf['topo_uuid'] == end_node_topo_uuid]

start_node = start_node_gdf.iloc[0]["geometry"]
end_node = end_node_gdf.iloc[0]["geometry"]

shortest_paths = OsmGt.shortest_path_from_location(
    "Roanne",
    [
        (start_node, end_node),
        (start_node, end_node),  # duplicate pairs are cleaned
    ],
    mode="pedestrian"
)
layers_to_add = [
    {
        "input_gdf": shortest_paths[["geometry"]],
        "legend": "shortest_path",
        "color": "red",
        "line_width": 5
    },
    {
        "input_gdf": start_node_gdf,
        "legend": "source node",
        "color": "blue",
        "style": "circle",
        "size": 9
    },
    {
        "input_gdf": end_node_gdf,
        "legend": "target node",
        "color": "green",
        "style": "circle",
        "size": 9
    },
]

my_shortest_path_map = Gdf2Bokeh(
    "My shortest path - from OsmGT (https://github.com/amauryval)",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)

display(shortest_paths)

## Compute an isochrone

### Isochrone from times

In [6]:
%%time


source_node1 = pois_gdf[pois_gdf['topo_uuid'] == 99]
source_node2 = pois_gdf[pois_gdf['topo_uuid'] == 167]

source_node_wkt1 = source_node1.iloc[0]["geometry"]
source_node_wkt2 = source_node2.iloc[0]["geometry"]

# 2 = 2 min ; 5 = 5 min ; 10 = 10 min
isochrones_polygon_values = {
    2: "#d9ef8b",
    5: "#fee08b",
    10: "#f46d43"
}

isochrones_lines_values = {
    2: "#668100",
    5: "#e2a803",
    10: "#962603"
}

trip_speed = 3  # km/h

isochrones_polygons, isochrones_lines = OsmGt.isochrone_from_source_node(
    [source_node_wkt1, source_node_wkt2],
    list(isochrones_polygon_values.keys()),
    trip_speed,
    mode="pedestrian"
)

isochrones_polygons["color"] = isochrones_polygons["iso_name"].map(lambda x: isochrones_polygon_values[x])
isochrones_lines["color"] = isochrones_lines["iso_name"].map(lambda x: isochrones_lines_values[x])


layers_to_add = [
    {
        "input_gdf": isochrones_polygons,
        "legend": "iso_name",
        "fill_color": "color",
        "line_color": "color",
        "fill_alpha": 0.5
    },
    {
        "input_gdf": source_node1,
        "legend": "Source node1",
        "style": "circle",
        "color": "red",
        "size": 5
    },
    {
        "input_gdf": source_node2,
        "legend": "Source node2",
        "style": "circle",
        "color": "red",
        "size": 5
    },
    {
        "input_gdf": isochrones_lines,
        "legend": "iso_name",
        "color": "color",
        "line_width": 2
    },
]

my_shortest_path_map = Gdf2Bokeh(
    "Isochrones from times - from OsmGT (https://github.com/amauryval)",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)

print("\nIsochrones polygons output")
display(isochrones_polygons)

print("\nIsochrones lines output")
display(isochrones_lines.head(2))


2021-04-21 17:41:59 - OsmGtIsochrone  - INFO     : Isochrone processing...
2021-04-21 17:41:59 - OsmGtIsochrone  - INFO     : From bbox: (4.064834150653223, 46.030644251224864, 4.078003049346776, 46.039846871081735)
2021-04-21 17:41:59 - OsmGtIsochrone  - INFO     : Loading data...
2021-04-21 17:42:15 - OsmGtIsochrone  - INFO     : OverpassApi: Query 504:Gateway Timeout in 15.73 sec.
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : OverpassApi: Query 200:OK in 4.52 sec.
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : Rebuild network data
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : Network cleaning...
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : Starting: Adding new nodes on the network
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : Find nearest line for each node
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : Split line
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO     : Topology lines checker: to add: 2, to split: 2
2021-04-21 17:42:23 - OsmGtIsochrone  - INFO

ERROR:bokeh.core.validation.check:E-1006 (NON_MATCHING_DATA_SOURCES_ON_LEGEND_ITEM_RENDERERS): LegendItem.label is a field, but renderer data sources don't match: LegendItem(id='1265', ...)



Isochrones polygons output


,iso_name,time_unit,iso_distance,distance_unit,geometry,id,color
0,10,minutes,500,meters,"POLYGON ((4.06809 46.03204, 4.06808 46.03204, ...",0,#f46d43
1,10,minutes,500,meters,"POLYGON ((4.07320 46.03403, 4.07320 46.03404, ...",1,#f46d43
2,5,minutes,250,meters,"POLYGON ((4.07138 46.03265, 4.07138 46.03265, ...",2,#fee08b
3,5,minutes,250,meters,"POLYGON ((4.06954 46.03619, 4.06967 46.03617, ...",3,#fee08b
4,2,minutes,100,meters,"POLYGON ((4.06960 46.03593, 4.06956 46.03599, ...",4,#d9ef8b
5,2,minutes,100,meters,"POLYGON ((4.07246 46.03536, 4.07255 46.03534, ...",5,#d9ef8b



Isochrones lines output


,highway,maxspeed,name,oneway,id,osm_url,topo_uuid,topology,junction,cycleway,...,bicycle,tunnel,horse,source:name,segregated,incline,iso_name,iso_distance,geometry,color
0,tertiary,NaN,Rue Raoul Follereau,yes,147487948,https://www.openstreetmap.org/way/147487948,100_00,split,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,500.0,"LINESTRING (4.07180 46.03265, 4.07180 46.03265...",#962603
1,tertiary,NaN,Rue Raoul Follereau,yes,147487948,https://www.openstreetmap.org/way/147487948,100_01,split,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,250.0,"LINESTRING (4.07172 46.03264, 4.07173 46.03264...",#e2a803


CPU times: user 23.5 s, sys: 455 ms, total: 23.9 s
Wall time: 47.9 s


In [8]:
print(
    len(isochrones_polygons)
)
print(
    len(isochrones_lines)
)
isochrones_lines.to_file("test.sh")


6
396


### Isochrone from distances

In [ ]:
%%time
start_node_topo_uuid = 47  # change the value here to define a new source point to compute isochrones
source_node = pois_gdf[pois_gdf['topo_uuid'] == start_node_topo_uuid]

trip_speed = 3  # km/h

isochrones_polygon_values = {
    250: "#d9ef8b",
    500: "#fee08b",
    750: "#f46d43",
    1000: "#8B4513"
}
# 250 = 250 meters ; 500 = 500 meters ; etc

isochrones_lines_values = {
    250: "#668100",
    500: "#e2a803",
    750: "#962603",
    1000: "#000000"
}

location_point = source_node.iloc[0]["geometry"]
isochrones_polygons, isochrones_lines = OsmGt.isochrone_distance_from_source_node(
    location_point,
    list(isochrones_polygon_values.keys()),  # meters
    trip_speed,
    mode="pedestrian"
)

isochrones_polygons["color"] = isochrones_polygons["iso_distance"].map(lambda x: isochrones_polygon_values[x])
isochrones_lines["color"] = isochrones_lines["iso_distance"].map(lambda x: isochrones_lines_values[x])


layers_to_add = [
    {
        "input_gdf": isochrones_polygons,
        "legend": "iso_distance",
        "fill_color": "color",
        "line_color": "color",
        "fill_alpha": 0.5
    },
    {
        "input_gdf": isochrones_lines,
        "legend": "iso_distance",
        "color": "color",
        "line_width": 2
    },
    {
        "input_gdf": source_node,
        "legend": "Source node",
        "style": "circle",
        "color": "red",
        "size": 15
    },
]

my_shortest_path_map = Gdf2Bokeh(
    "Isochrones from distance - from OsmGT (https://github.com/amauryval)",
    layers=layers_to_add
)
show(my_shortest_path_map.figure)
